In [1]:
import backtrader as bt
import yfinance as yf
import numpy as np
from scipy.optimize import linprog
import IPython

In [2]:

# Bayes' Theorem to update the probability of mean reversion
def bayesian_update(prior_prob, likelihood, marginal_prob):
    if marginal_prob == 0:
        return 0
    return (likelihood * prior_prob) / marginal_prob

# Function to optimize position size using Linear Programming
def optimize_position(prices, capital, risk_factor):
    n = len(prices)
    # Coefficients for the objective function (negative for maximization)
    c = -np.array(prices)

    # Inequality constraint matrix
    A = np.array([[1] * n])
    # Inequality constraint bounds
    b = [capital * risk_factor]

    # Bounds for each position (no short selling)
    x_bounds = [(0, None) for _ in range(n)]

    # Solve the linear programming problem
    res = linprog(c, A_ub=A, b_ub=b, bounds=x_bounds, method='highs')
    
    if res.success:
        return res.x  # Optimal positions
    else:
        return np.zeros(n)  # No valid solution found

class BayesianLPMeanReversionStrategy(bt.Strategy):
    params = (
        ('period', 20),
        ('devfactor', 2.0),
        ('reversal_prob', 0.5),
        ('signal_strength', 0.7),
        ('signal_prob', 0.6),
        ('risk_factor', 0.1),  # Risk factor for LP
    )

    def __init__(self):
        self.sma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.period)
        self.stddev = bt.indicators.StandardDeviation(self.data.close, period=self.params.period)
        self.upper_band = self.sma + self.stddev * self.params.devfactor
        self.lower_band = self.sma - self.stddev * self.params.devfactor
        self.buy_signal = self.data.close < self.lower_band
        self.sell_signal = self.data.close > self.upper_band
        self.buy_orders = []
        self.sell_orders = []

    def next(self):
        if not self.position:
            buy_prob = bayesian_update(self.params.reversal_prob, self.params.signal_strength, self.params.signal_prob)
            if self.buy_signal and buy_prob > 0.6:
                # Optimize position size using Linear Programming
                prices = [self.data.close[0]]  # Current price
                capital = self.broker.getvalue()  # Available capital
                positions = optimize_position(prices, capital, self.params.risk_factor)
                
                if positions[0] > 0:
                    self.buy(size=int(positions[0]))
                    print(f"Buy Signal: Updated Buy Probability = {buy_prob:.2f}, Position Size = {positions[0]}")

        elif self.position:
            sell_prob = bayesian_update(self.params.reversal_prob, self.params.signal_strength, self.params.signal_prob)
            if self.sell_signal and sell_prob > 0.6:
                prices = [self.data.close[0]]
                capital = self.broker.getvalue()
                positions = optimize_position(prices, capital, self.params.risk_factor)
                
                if positions[0] > 0:
                    self.sell(size=int(positions[0]))
                    print(f"Sell Signal: Updated Sell Probability = {sell_prob:.2f}, Position Size = {positions[0]}")

# Function to fetch historical data using yfinance
def get_historical_data(symbol, start, end):
    data = yf.download(symbol, start=start, end=end)
    return data

In [18]:
# Create and set up the backtest environment
def run_bayesian_lp_backtest(symbol='AAPL', start='2022-01-01', end='2023-01-01'):
    data = get_historical_data(symbol, start, end)
    data_feed = bt.feeds.PandasData(dataname=data)

    cerebro = bt.Cerebro()
    cerebro.addstrategy(BayesianLPMeanReversionStrategy)
    cerebro.adddata(data_feed)
    cerebro.broker.setcash(100000.0)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="sharpe")
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trades")

    results = cerebro.run()
    sharpe_ratio = results[0].analyzers.sharpe.get_analysis()
    drawdown = results[0].analyzers.drawdown.get_analysis()
    trades = results[0].analyzers.trades.get_analysis()

    #print(f"Sharpe Ratio: {sharpe_ratio['sharperatio']:.2f}")
    print(f"Max Drawdown: {drawdown.max.drawdown:.2f}%")
    print(f"Total Trades: {trades.total.total}")
    print(f"Final Portfolio Value: {cerebro.broker.getvalue():.2f}")

    #cerebro.plot()

In [20]:
# Run the backtest for AAPL from 2022-01-01 to 2023-01-01
if __name__ == "__main__":
    run_bayesian_lp_backtest(symbol='AAPL', start='2020-01-01', end='2023-01-01')

[*********************100%%**********************]  1 of 1 completed


Max Drawdown: 0.00%
Total Trades: 0
Final Portfolio Value: 100000.00


In [16]:
get_historical_data("AAPL", start="2022-01-01", end="2023-01-01")

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,177.830002,182.880005,177.710007,182.009995,179.273636,104487900
2022-01-04,182.630005,182.940002,179.119995,179.699997,176.998337,99310400
2022-01-05,179.610001,180.169998,174.639999,174.919998,172.290207,94537600
2022-01-06,172.699997,175.300003,171.639999,172.000000,169.414108,96904000
2022-01-07,172.889999,174.139999,171.029999,172.169998,169.581604,86709100
...,...,...,...,...,...,...
2022-12-23,130.919998,132.419998,129.639999,131.860001,130.631363,63814900
2022-12-27,131.380005,131.410004,128.720001,130.029999,128.818405,69007800
2022-12-28,129.669998,131.029999,125.870003,126.040001,124.865585,85438400


In [13]:
!pip install --upgrade yfinance

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.41
    Uninstalling yfinance-0.2.41:
      Successfully uninstalled yfinance-0.2.41
